# Experiment 1: Run Cross-Agent Evaluations

This notebook runs MCP evaluations across three coding agents: Claude Code, Goose, and Gemini.

**Each evaluation takes 2-3 hours** and tests 25 cases × 4 MCP servers = 100 evaluations per agent.

---

## Setup

In [ ]:
import subprocess
import os
from pathlib import Path
from datetime import datetime

# Set working directory
project_root = Path.cwd().parent
os.chdir(project_root)
print(f"Working directory: {os.getcwd()}")

## 1. Run Claude Code Evaluation

**Command:** `./run_claude_eval.sh`

**Duration:** ~2-3 hours

**Output:** `results/compare_agents/claude_YYYYMMDD.yaml`

In [ ]:
# Check if Claude evaluation already exists
claude_result = f"results/compare_agents/claude_{datetime.now().strftime('%Y%m%d')}.yaml"
if os.path.exists(claude_result):
    print(f"✓ Claude evaluation already completed: {claude_result}")
else:
    print("Running Claude evaluation (this takes 2-3 hours)...")
    result = subprocess.run(["./run_claude_eval.sh"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✓ Claude evaluation complete!")
    else:
        print(f"✗ Error: {result.stderr}")

## 2. Run Goose Evaluation

**Command:** `./run_goose_eval.sh`

**Duration:** ~2-3 hours

**Output:** `results/compare_agents/goose_YYYYMMDD.yaml`

In [ ]:
# Check if Goose evaluation already exists
goose_result = f"results/compare_agents/goose_{datetime.now().strftime('%Y%m%d')}.yaml"
if os.path.exists(goose_result):
    print(f"✓ Goose evaluation already completed: {goose_result}")
else:
    print("Running Goose evaluation (this takes 2-3 hours)...")
    result = subprocess.run(["./run_goose_eval.sh"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✓ Goose evaluation complete!")
    else:
        print(f"✗ Error: {result.stderr}")

## 3. Run Gemini Evaluation

**Command:** `./run_gemini_eval.sh`

**Duration:** ~2-3 hours

**Output:** `results/compare_agents/gemini_YYYYMMDD.yaml`

In [ ]:
# Check if Gemini evaluation already exists
gemini_result = f"results/compare_agents/gemini_{datetime.now().strftime('%Y%m%d')}.yaml"
if os.path.exists(gemini_result):
    print(f"✓ Gemini evaluation already completed: {gemini_result}")
else:
    print("Running Gemini evaluation (this takes 2-3 hours)...")
    result = subprocess.run(["./run_gemini_eval.sh"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✓ Gemini evaluation complete!")
    else:
        print(f"✗ Error: {result.stderr}")

## Check Evaluation Status

In [ ]:
import yaml
from glob import glob

# Find all result files
result_files = sorted(glob('results/compare_agents/*.yaml'))

print("Available evaluation results:")
print("=" * 70)

for f in result_files:
    try:
        with open(f, 'r') as file:
            data = yaml.safe_load(file)
            if data and 'results' in data:
                agent = Path(f).stem.split('_')[0]
                date = Path(f).stem.split('_')[1]
                count = len(data['results'])
                summary = data.get('summary', {})
                pass_rate = summary.get('pass_rate', 0) * 100 if summary else 0
                
                print(f"\n{agent.upper()} ({date}):")
                print(f"  File: {f}")
                print(f"  Tests: {count}")
                print(f"  Pass rate: {pass_rate:.1f}%")
    except Exception as e:
        print(f"\nError reading {f}: {e}")

print("\n" + "=" * 70)

## Run Analysis

Once all evaluations are complete, run the cross-agent analysis:

In [ ]:
# Check if all evaluations are complete, then run analysis
result_files = glob('results/compare_agents/*.yaml')
agents_complete = {'claude': False, 'goose': False, 'gemini': False}

for f in result_files:
    agent = Path(f).stem.split('_')[0]
    if agent in agents_complete:
        agents_complete[agent] = True

if all(agents_complete.values()):
    print("All evaluations complete! Running cross-agent analysis...")
    result = subprocess.run([
        "jupyter", "nbconvert", "--execute", "--to", "notebook", 
        "--inplace", "experiment_1_cross_agent_analysis.ipynb"
    ], capture_output=True, text=True)
    if result.returncode == 0:
        print("✓ Analysis complete!")
    else:
        print(f"✗ Error: {result.stderr}")
else:
    missing = [k for k, v in agents_complete.items() if not v]
    print(f"⏳ Waiting for evaluations to complete: {', '.join(missing)}")
    print("Run this cell again after all evaluations finish.")

---

## Configuration Details

### MCP Servers Tested

1. **ARTL MCP** - Berkeley Lab Contextualizer AI
   - Repo: https://github.com/contextualizer-ai/artl-mcp
   - Command: `uvx artl-mcp`

2. **Simple PubMed MCP**
   - Repo: https://github.com/andybrandt/mcp-simple-pubmed
   - Command: `uvx mcp-simple-pubmed`
   - Env: `PUBMED_EMAIL=justinreese@lbl.gov`

3. **BioMCP**
   - Repo: https://github.com/genomoncology/biomcp
   - Command: `uv run --with biomcp-python biomcp run`

4. **PubMed MCP** (chrismannina)
   - Repo: https://github.com/chrismannina/pubmed-mcp
   - Command: `uv run --with git+https://github.com/chrismannina/pubmed-mcp@main -m src.main`
   - Env: `PUBMED_API_KEY`, `PUBMED_EMAIL=justinreese@lbl.gov`

### Test Cases

25 literature retrieval tasks organized into groups:
- Text extraction (9 cases)
- Metadata (8 cases)  
- Table/Figure/Legend extraction (4 cases)
- Summarization (2 cases)
- Supplementary material (1 case)
- Publication status (1 case)

### Evaluation Framework

- **Framework:** Metacoder (https://github.com/ai4curation/metacoder)
- **Metric:** CorrectnessMetric (semantic similarity)
- **Pass threshold:** 0.9
- **Total evaluations per agent:** 100 (25 cases × 4 MCPs)

### References

- **MCP Server Catalog:** https://docs.google.com/spreadsheets/d/1506RuqfyUrBHd6lGNtY5j688CvVwk5mf5h_LEfsPIp4/edit?gid=614919216#gid=614919216
- **Experiment Design:** `notes/experiment_1_cross_agent_comparison.md`
- **Results Documentation:** `notes/EXPERIMENT_1_RESULTS.md`